In [2]:
import torch
from transformers import LongformerTokenizer, LongformerForSequenceClassification, AutoTokenizer, AutoModelForSequenceClassification
import unidecode
import contractions
import sys
sys.path.append('..')

In [3]:
from attention_model.PrepareDf import *

In [4]:
transcript_pt = '/Users/akshaykekuda/Desktop/CSR-SA/manual_score_transcriptions/single_ch/'

In [5]:
path_to_handscored_p ='/Users/akshaykekuda/Desktop/CSR-SA/attention_model/ScoringDetail_viw_all_subscore.p'

In [6]:
sub_score_categories = ['Cross Selling', 'Creates Incentive', 'Education', 'Processes', 'Product Knowledge', 'Greeting', 'Professionalism', 'Confidence',  'Retention',
                        'Documentation']
scoring_criteria = sub_score_categories[:4]

In [7]:
score_df, q_text = prepare_score_df(
    path_to_handscored_p, workgroup='all')
train_df = prepare_trancript_score_df(score_df, q_text, transcript_pt, None)

Dataframe creation done


100%|██████████| 14081/14081 [00:11<00:00, 1251.36it/s]

Number of Calls = 13890


In [8]:
def post_process(arr):
    arr = [txt.capitalize() for txt in arr]
    return '. '.join(arr)

In [9]:
train_df.text = train_df.text.apply(lambda x: post_process(x))

In [10]:
from datasets import Dataset

In [11]:
train_ds = Dataset.from_pandas(train_df)

In [12]:
# checkpoint='allenai/longformer-base-4096'
checkpoint = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint, )
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=8)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

In [13]:
def preprocess_function(example):
    new_dict = tokenizer(example["text"], truncation=True)
    new_dict['labels'] = [example[criteria] for criteria in scoring_criteria]
    return new_dict

In [14]:
cols = train_ds.column_names
tokenized_ds = train_ds.map(preprocess_function, remove_columns=cols)

  0%|          | 0/13890 [00:00<?, ?ex/s]

In [15]:
tokenized_ds.set_format('torch')

In [16]:
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments
data_collator = DataCollatorWithPadding(tokenizer, padding=True, return_tensors='pt')

In [33]:
class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fn = torch.nn.CrossEntropyLoss()
        for i in range(self.model.config.num_labels//2):
            loss = loss_fn(outputs[0][:, 2*i:2*(i+1)], labels[:, i])

        return (loss, outputs) if return_outputs else loss

In [34]:
training_args = TrainingArguments(
    output_dir="test_trainer",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    report_to='wandb'
)

PyTorch: setting up devices


In [35]:
trainer = MultilabelTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds,
    data_collator = data_collator,
    tokenizer=tokenizer
)

In [36]:
trainer.train()

***** Running training *****
  Num examples = 13890
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 869
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss


KeyboardInterrupt: 